# Install pachctl

In [1]:
!curl -L https://github.com/pachyderm/pachyderm/releases/download/v2.7.5/pachctl_2.7.5_linux_amd64.tar.gz | tar -xzv --strip-components=1 -C /usr/local/bin

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
pachctl_2.7.5_linux_amd64/pachctl
100 40.4M  100 40.4M    0     0  27.6M      0  0:00:01  0:00:01 --:--:-- 47.4M


# Connect to the cluster

In [2]:
cluster_address = "grpc://34.90.36.178:80"

!pachctl connect $cluster_address
!pachctl auth login

New context 'grpc://34.90.36.178:80' created, will connect to Pachyderm at grpc://34.90.36.178:80
Context 'grpc://34.90.36.178:80' set as active
2023-11-22T16:04:46.068Z	ERROR	client/client.go:618	WARNING: Could not inspect project "default": error getting permissions for project "default": cannot get authenticated user: rpc error: code = Unauthenticated desc = no authentication token (try logging in)
You will momentarily be directed to your IdP and asked to authorize Pachyderm's login app on your IdP.

Paste the following URL into a browser if not automatically redirected:

http://34.90.36.178.nip.io/dex/auth?client_id=pachd&nonce=uwOrEPV4Ey2xg0JyjaPkBRIH-5PCNyc&redirect_uri=http%3A%2F%2F34.90.36.178.nip.io%2Fauthorization-code%2Fcallback&response_type=code&scope=email+profile+groups+openid&state=1-o0xY2Wu-XgmihGx7XGn4gjIQ2MlM8


Couldn't open a browser, visit the page manually.
Retrieving Pachyderm token...


Open the link in your browser and enter your credentials

# Create your MLDM project

In [3]:
project_name = "denisd-lab" # change your project name

In [4]:
!pachctl create project $project_name

In [6]:
!pachctl list projects # default should be the active one

ACTIVE PROJECT        ACCESS_LEVEL                 CREATED            DESCRIPTION
*      default        [clusterAdmin projectWriter] 3 weeks ago        -
       my_project     [clusterAdmin projectOwner]  3 weeks ago        -
       my_project2    [clusterAdmin projectOwner]  2 weeks ago        -
       andrea_project [clusterAdmin projectOwner]  About a minute ago -
       bv             [clusterAdmin projectOwner]  58 seconds ago     -
       project-tb     [clusterAdmin projectOwner]  56 seconds ago     -
       denisd-lab     [clusterAdmin projectOwner]  46 seconds ago     -
       tp_project     [clusterAdmin projectOwner]  27 seconds ago     -


# Set the context to your project

In [7]:
!pachctl config update context --project $project_name

editing the currently active context "grpc://34.90.36.178:80"


In [8]:
!pachctl list projects # your project should be the active one

ACTIVE PROJECT        ACCESS_LEVEL                 CREATED            DESCRIPTION
       default        [clusterAdmin projectWriter] 3 weeks ago        -
       my_project     [clusterAdmin projectOwner]  3 weeks ago        -
       my_project2    [clusterAdmin projectOwner]  2 weeks ago        -
       andrea_project [clusterAdmin projectOwner]  About a minute ago -
       bv             [clusterAdmin projectOwner]  About a minute ago -
       project-tb     [clusterAdmin projectOwner]  About a minute ago -
*      denisd-lab     [clusterAdmin projectOwner]  About a minute ago -
       tp_project     [clusterAdmin projectOwner]  About a minute ago -


In [9]:
!pachctl list repos # empty

PROJECT NAME CREATED SIZE (MASTER) DESCRIPTION 


# Create the first repo and upload data

In [10]:
!pachctl create repo raw_videos_and_images

In [11]:
!pachctl list repos # raw_videos_and_images repo should be listed here

PROJECT    NAME                  CREATED       SIZE (MASTER) ACCESS_LEVEL                          DESCRIPTION 
denisd-lab raw_videos_and_images 2 seconds ago ≤ 0B          [clusterAdmin projectOwner repoOwner]             


In [12]:
!pachctl put file raw_videos_and_images@master:liberty.png -f files/liberty.jpg
!pachctl put file raw_videos_and_images@master:cat-sleeping.MOV -f files/cat-sleeping.MOV
!pachctl put file raw_videos_and_images@master:robot.png -f files/robot.jpg
!pachctl put file raw_videos_and_images@master:highway.MOV -f files/highway.MOV

files/liberty.jpg 133.86 KB / 133.86 KB [==========================] 0s 0.00 b/s
files/liberty.jpg 133.86 KB / 133.86 KB [==========================] 0s 0.00 b/s
files/cat-sleeping.MOV 12.96 MB / 12.96 MB [=======================] 0s 0.00 b/s
files/cat-sleeping.MOV 12.96 MB / 12.96 MB [=======================] 0s 0.00 b/s
files/cat-sleeping.MOV 12.96 MB / 12.96 MB [=======================] 0s 0.00 b/s
files/cat-sleeping.MOV 12.96 MB / 12.96 MB [=======================] 0s 0.00 b/s
files/cat-sleeping.MOV 12.96 MB / 12.96 MB [=======================] 0s 0.00 b/s
files/cat-sleeping.MOV 12.96 MB / 12.96 MB [=======================] 0s 0.00 b/s
files/robot.jpg 123.23 KB / 123.23 KB [============================] 0s 0.00 b/s
files/robot.jpg 123.23 KB / 123.23 KB [============================] 0s 0.00 b/s
files/robot.jpg 123.23 KB / 123.23 KB [============================] 0s 0.00 b/s
files/highway.MOV 9.98 MB / 9.98 MB [==============================] 0s 0.00 b/s
files/highway.MOV 9.98 MB / 

In [13]:
!pachctl list files raw_videos_and_images@master

NAME              TYPE SIZE     
/cat-sleeping.MOV file 12.36MiB 
/highway.MOV      file 9.516MiB 
/liberty.png      file 130.7KiB 
/robot.png        file 120.3KiB 


# Creating the video mp4 converter pipeline

In [14]:
!cat video_mp4_converter.yaml 

pipeline:
  name: video_mp4_converter
input:
  pfs:
    repo: raw_videos_and_images
    glob: "/*"
transform:
  image: lbliii/video_mp4_converter:1.0.14
  cmd:
    - python3
    - /video_mp4_converter.py
    - --input
    - /pfs/raw_videos_and_images/
    - --output
    - /pfs/out/
autoscaling: true


In [15]:
!pachctl create pipeline -f video_mp4_converter.yaml

# Datums and glob patterns

In [16]:
# how many datums does the video mp4 converter pipeline see?
!pachctl list datum -f video_mp4_converter.yaml

ID                                                               FILES                                                                               STATUS TIME               
34c72adefd73aa41cf20fc4003532c1a29c16b02433c492de9a8dc2422d19695 denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/robot.png        -      Less than a second 
48445a6c755bacf3baf96b9be9cf0634f18d09e3daae8b672ef4cc5509df4ae3 denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/liberty.png      -      Less than a second 
b674bbcacca58f7b48ce78b6919590f333688e888294f5ca6ded9646ccfa9403 denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/highway.MOV      -      Less than a second 
dbf90651d6bd62739fdae0125cc82e082733d105613d2ecf8da3af9098efbf8e denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/cat-sleeping.MOV -      Less than a second 


In [17]:
# what if its glob pattern was "/"?
!pachctl glob file raw_videos_and_images@master:/

NAME TYPE SIZE     
/    dir  22.12MiB 


In [18]:
# what if its glob pattern was "/*"?
!pachctl glob file raw_videos_and_images@master:/*

NAME              TYPE SIZE     
/cat-sleeping.MOV file 12.36MiB 
/highway.MOV      file 9.516MiB 
/liberty.png      file 130.7KiB 
/robot.png        file 120.3KiB 


In [19]:
# what if its glob pattern was "/*.png"?
!pachctl glob file raw_videos_and_images@master:/*.png

NAME         TYPE SIZE     
/liberty.png file 130.7KiB 
/robot.png   file 120.3KiB 


In [20]:
# what if its glob pattern was "/*.mp4"?
!pachctl glob file raw_videos_and_images@master:/*.mp4

NAME TYPE SIZE 


# Updating the video mp4 converter pipeline

Edit __video_mp4_converter.yaml__ to only process .MOV files and then, update the pipeline with:

In [21]:
!pachctl update pipeline -f video_mp4_converter.yaml

In [23]:
!pachctl list job

ID                               SUBJOBS PROGRESS CREATED        MODIFIED
cc2e3d1fcf854480895af487a27a0714 1       ▇▇▇▇▇▇▇▇ 2 minutes ago  2 minutes ago  
217694b51c444890929bc6d1d953d47d 1       ▇▇▇▇▇▇▇▇ 19 minutes ago 19 minutes ago 


In [24]:
!pachctl list commit # notice that some commit IDs will be identical to job IDs

ID                               SUBCOMMITS PROGRESS CREATED        MODIFIED       
cc2e3d1fcf854480895af487a27a0714 4          ▇▇▇▇▇▇▇▇ 35 minutes ago 2 minutes ago  
217694b51c444890929bc6d1d953d47d 4          ▇▇▇▇▇▇▇▇ 35 minutes ago 19 minutes ago 
4bf1694b1d9e422dbe13ed103c82f1f1 1          ▇▇▇▇▇▇▇▇ 35 minutes ago 35 minutes ago 
e9cf39575158423298cb8b5541ff5caf 1          ▇▇▇▇▇▇▇▇ 35 minutes ago 35 minutes ago 
e5c1ee24c85f4f6fa2cfdb0994ac31f4 1          ▇▇▇▇▇▇▇▇ 35 minutes ago 35 minutes ago 
dc219964fd334023913be199c75b5bb1 1          ▇▇▇▇▇▇▇▇ 35 minutes ago 35 minutes ago 


# Inspecting jobs, logs and pipeline spec

In [25]:
!pachctl list pipeline --spec --output yaml # gives the entire list of pipelines, corresponds to the latest commit by default

autoscaling: true
datum_tries: "3"
input:
  pfs:
    branch: master
    glob: /*.MOV
    name: raw_videos_and_images
    project: denisd-lab
    repo: raw_videos_and_images
    repo_type: user
output_branch: master
pipeline:
  name: video_mp4_converter
  project:
    name: denisd-lab
reprocess_spec: until_success
salt: 1918f22cdfa8473e8224c92b96f86136
transform:
  cmd:
    - python3
    - /video_mp4_converter.py
    - --input
    - /pfs/raw_videos_and_images/
    - --output
    - /pfs/out/
  image: lbliii/video_mp4_converter:1.0.14


In [27]:
!pachctl inspect job video_mp4_converter@cc2e3d1fcf854480895af487a27a0714  # add a job id to inspect the job

ID: cc2e3d1fcf854480895af487a27a0714
Pipeline: video_mp4_converter
Project: denisd-lab
Started: 11 minutes ago 
Duration: 2 seconds 
State: success
Reason: 
Processed: 0
Failed: 0
Skipped: 2
Recovered: 0
Total: 2
Data Downloaded: 0B
Data Uploaded: 0B
Download Time: Less than a second
Process Time: Less than a second
Upload Time: Less than a second
Datum Timeout: <nil>
Job Timeout: <nil>
Worker Status:
WORKER              JOB                 DATUM               STARTED             
Restarts: 0
ParallelismSpec: <nil>
Input: {
  "pfs": {
    "project": "denisd-lab",
    "name": "raw_videos_and_images",
    "repo": "raw_videos_and_images",
    "repo_type": "user",
    "branch": "master",
    "commit": "cc2e3d1fcf854480895af487a27a0714",
    "glob": "/*.MOV"
  }
}
Transform: {
  "image": "lbliii/video_mp4_converter:1.0.14",
  "cmd": [
    "python3",
    "/video_mp4_converter.py",
    "--input",
    "/pfs/raw_videos_and_images/",
    "--output",
    "/pfs/out/"
  ]
} 
Output Commit: cc2e3d1f

In [28]:
!pachctl logs --pipeline video_mp4_converter # should be empty as datums were skipped when updating the pipeline spec

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Converting video: /pfs/raw_videos_and_images/cat-sleeping.MOV
Converted 154 frames
Converted: cat-sleeping.MOV to /pfs/out/cat-sleeping.mp4
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Converting video: /pfs/raw_videos_and_images/highway.MOV
Converted 156 frames
Converted: highway.MOV to /pfs/out/highway.mp4
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Skipping: /pfs/raw_videos_and_images/liberty.png
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Skipping: /pfs/raw_videos_and_images/robot.png


In [30]:
!pachctl logs --job video_mp4_converter@217694b51c444890929bc6d1d953d47d  # add a job id to get logs from the job

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Converting video: /pfs/raw_videos_and_images/cat-sleeping.MOV
Converted 154 frames
Converted: cat-sleeping.MOV to /pfs/out/cat-sleeping.mp4
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Converting video: /pfs/raw_videos_and_images/highway.MOV
Converted 156 frames
Converted: highway.MOV to /pfs/out/highway.mp4
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Skipping: /pfs/raw_videos_and_images/liberty.png
Input: /pfs/raw_videos_and_images/
Output: /pfs/out/
Skipping: /pfs/raw_videos_and_images/robot.png


# Updating the video mp4 converter pipeline and reprocess your datums

In [ ]:
!pachctl update pipeline -f video_mp4_converter.yaml --reprocess

# Creating the image flattener pipeline

In [31]:
!cat image_flattener.yaml 

pipeline:
  name: image_flattener
input:
  pfs:
    repo: video_mp4_converter
    glob: "/*"
transform:
  image: lbliii/image_flattener:1.0.0
  cmd:
    - python3
    - /image_flattener.py
    - --input
    - /pfs/video_mp4_converter
    - --output
    - /pfs/out/
autoscaling: true


In [32]:
!pachctl create pipeline -f image_flattener.yaml

In [33]:
!pachctl list pipeline --spec --output yaml

autoscaling: true
datum_tries: "3"
input:
  pfs:
    branch: master
    glob: /*
    name: video_mp4_converter
    project: denisd-lab
    repo: video_mp4_converter
    repo_type: user
output_branch: master
pipeline:
  name: image_flattener
  project:
    name: denisd-lab
reprocess_spec: until_success
salt: 047f8a49439146088e031f6517cdb91f
transform:
  cmd:
    - python3
    - /image_flattener.py
    - --input
    - /pfs/video_mp4_converter
    - --output
    - /pfs/out/
  image: lbliii/image_flattener:1.0.0
---
autoscaling: true
datum_tries: "3"
input:
  pfs:
    branch: master
    glob: /*.MOV
    name: raw_videos_and_images
    project: denisd-lab
    repo: raw_videos_and_images
    repo_type: user
output_branch: master
pipeline:
  name: video_mp4_converter
  project:
    name: denisd-lab
reprocess_spec: until_success
salt: 1918f22cdfa8473e8224c92b96f86136
transform:
  cmd:
    - python3
    - /video_mp4_converter.py
    - --input
    - /pfs/raw_videos_and_images/
    - --output
   

# Create the image tracer pipeline

In [34]:
!cat image_tracer.yaml

pipeline:
  name: image_tracer
description: A pipeline that performs image edge detection by using the OpenCV library.
input:
  union:
    - pfs:
        repo: raw_videos_and_images
        glob: "/*.{png,jpg,jpeg}"
    - pfs:
        repo: image_flattener
        glob: "/*"
transform:
  image: lbliii/image_tracer:1.0.8
  cmd:
    - python3
    - /image_tracer.py
    - --input
    - /pfs/raw_videos_and_images
    - /pfs/image_flattener
    - --output
    - /pfs/out/
autoscaling: true


In [35]:
!pachctl list datum -f image_tracer.yaml

ID                                                               FILES                                                                          STATUS TIME               
34c72adefd73aa41cf20fc4003532c1a29c16b02433c492de9a8dc2422d19695 denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/robot.png   -      Less than a second 
48445a6c755bacf3baf96b9be9cf0634f18d09e3daae8b672ef4cc5509df4ae3 denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/liberty.png -      Less than a second 
766def4e90afaff44c357f0920c1188468628b535e261a95897459712c8b2af1 denisd-lab/image_flattener@a228fa11a14641859fdff081208bc289:/highway/          -      Less than a second 
96d411d2e0f6da9286417116436df4bb5d507046f788d0b3930ba51b4a36c41b denisd-lab/image_flattener@a228fa11a14641859fdff081208bc289:/cat-sleeping/     -      Less than a second 


In [38]:
!pachctl create pipeline -f image_tracer.yaml

# Create the movie gifer pipeline

In [36]:
!cat movie_gifer.yaml

pipeline:
  name: movie_gifer
description: A pipeline that converts frames into a gif using the OpenCV library.
input:
  union:
    - pfs:
        repo: image_flattener
        glob: "/*/"
    - pfs:
        repo: image_tracer
        glob: "/*/"
transform:
  image: lbliii/movie_gifer:1.0.5
  cmd:
    - python3
    - /movie_gifer.py
    - --input
    - /pfs/image_flattener
    - /pfs/image_tracer
    - --output
    - /pfs/out/
autoscaling: true


In [39]:
!pachctl list datum -f movie_gifer.yaml

ID                                                               FILES                                                                       STATUS TIME               
0b50bf7740d03bd95f7afdcda92d4e29505cc375224a09bd0b0cb44e17986f02 denisd-lab/image_tracer@dfc0fe66675541328a0ce8a0ee27a2e8:/cat-sleeping/     -      Less than a second 
61130e22df8ad523c2fa09a2ef8d6063806628c2e6ab5ba9ef52f0a288307632 denisd-lab/image_tracer@dfc0fe66675541328a0ce8a0ee27a2e8:/liberty_edges.png -      Less than a second 
766def4e90afaff44c357f0920c1188468628b535e261a95897459712c8b2af1 denisd-lab/image_flattener@a228fa11a14641859fdff081208bc289:/highway/       -      Less than a second 
96d411d2e0f6da9286417116436df4bb5d507046f788d0b3930ba51b4a36c41b denisd-lab/image_flattener@a228fa11a14641859fdff081208bc289:/cat-sleeping/  -      Less than a second 
c43349b0763d690c1143777ffe6167e1ea3804384213bd4910d9ecc8359a8b78 denisd-lab/image_tracer@dfc0fe66675541328a0ce8a0ee27a2e8:/highway/          -      Less than a 

In [40]:
!pachctl create pipeline -f movie_gifer.yaml # will take a few minutes to run

# Create the content shuffler pipeline

In [41]:
!cat content_shuffler.yaml

pipeline:
  name: content_shuffler
  description: A pipeline that collapses our inputs into one datum for the collager.
input:
  union:
    - pfs:
        repo: movie_gifer
        glob: "/"
    - pfs:
        repo: raw_videos_and_images
        glob: "/*.{png,jpg,jpeg}"
    - pfs:
        repo: image_tracer
        glob: "/*.{png,jpg,jpeg}"

transform:
  image: lbliii/content_shuffler:1.0.0
  cmd:
    - python3
    - /content_shuffler.py
    - --input
    - /pfs/movie_gifer
    - /pfs/raw_videos_and_images
    - /pfs/image_tracer
    - --output
    - /pfs/out/
autoscaling: true


In [42]:
!pachctl list datum -f content_shuffler.yaml

ID                                                               FILES                                                                          STATUS TIME               
34c72adefd73aa41cf20fc4003532c1a29c16b02433c492de9a8dc2422d19695 denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/robot.png   -      Less than a second 
48445a6c755bacf3baf96b9be9cf0634f18d09e3daae8b672ef4cc5509df4ae3 denisd-lab/raw_videos_and_images@4bf1694b1d9e422dbe13ed103c82f1f1:/liberty.png -      Less than a second 
61130e22df8ad523c2fa09a2ef8d6063806628c2e6ab5ba9ef52f0a288307632 denisd-lab/image_tracer@dfc0fe66675541328a0ce8a0ee27a2e8:/liberty_edges.png    -      Less than a second 
a33c434dbeb637c2c068c55dcfb8c91511bd433cf6b949993cc4933d937f105b denisd-lab/movie_gifer@fd587560d6db476ebf68da6e81960a5d:/                      -      Less than a second 
c80bc94ff21fd4fbb2f0bb43691cf10232c13e2c1d28cff649161c0b0db60242 denisd-lab/image_tracer@dfc0fe66675541328a0ce8a0ee27a2e8:/robot_edges.png      -

In [43]:
!pachctl create pipeline -f content_shuffler.yaml

# Create the content collager pipeline

In [44]:
!cat content_collager.yaml

pipeline:
  name: content_collager
  description: A pipeline that creates a static HTML collage.
input:
  pfs:
    glob: "/"
    repo: content_shuffler


transform:
  image: lbliii/content_collager:1.0.64
  cmd:
    - python3
    - /content_collager.py
    - --input
    - /pfs/content_shuffler
    - --output
    - /pfs/out/
autoscaling: true


In [45]:
!pachctl list datum -f content_collager.yaml

ID                                                               FILES                                                          STATUS TIME               
d426c304cd626dbb5bc09e228acd5d90be2b7faf396867ae381b91244ebe4c8c denisd-lab/content_shuffler@50a7eda1d6cf4e818284782058f77986:/ -      Less than a second 


In [46]:
!pachctl create pipeline -f content_collager.yaml

# Create the apache serving pipeline

In [47]:
!cat apache_serve.yaml

{
  "pipeline": { "name": "apache_serve"},
  "description": "Serve HTML Page on Apache",
  "transform": {
    "cmd": ["/bin/sh"],
    "stdin": ["cp -r /pfs/in/* /usr/local/apache2/htdocs/"],
    "image": "httpd:2.4"
  },
  "input": {
    "pfs": {
      "name": "in",
      "repo": "content_collager",
      "branch": "master",
      "glob": "/"
    }
  },
  "service": {
    "external_port" : 8080,
    "internal_port" : 80,
    "type" : "LoadBalancer"
  },
  "parallelism_spec" : {"constant": 1}
}

In [48]:
!pachctl list datum -f apache_serve.yaml

ID                                                               FILES                                                          STATUS TIME               
2ecb4c4b1e919d7efe4841d236aaac69d97ce4c5962b1fd96cd103dfc134b20a denisd-lab/content_collager@7bb280219c0944a1bc92602bd651f37f:/ -      Less than a second 


In [57]:
!pachctl create pipeline -f apache_serve.yaml

# Branches

## Create a new branch, and have it identical to master

In [50]:
!pachctl list branch raw_videos_and_images

BRANCH HEAD                             TRIGGER 
master 4bf1694b1d9e422dbe13ed103c82f1f1 -       


In [51]:
!pachctl create branch raw_videos_and_images@new-branch # empty branch

In [52]:
!pachctl list branch raw_videos_and_images

BRANCH     HEAD                             TRIGGER 
new-branch f82f22f9db9d42459f3adafaa5d53c8c -       
master     4bf1694b1d9e422dbe13ed103c82f1f1 -       


In [53]:
!pachctl create branch raw_videos_and_images@new-branch --head raw_videos_and_images@master # new-branch will be identical to master

In [54]:
!pachctl list branch raw_videos_and_images

BRANCH     HEAD                             TRIGGER 
new-branch 4bf1694b1d9e422dbe13ed103c82f1f1 -       
master     4bf1694b1d9e422dbe13ed103c82f1f1 -       


## Update the new branch

In [55]:
!pachctl put file raw_videos_and_images@new-branch:robot1.png -f "files/robot(1).jpg"

files/robot(1).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s
files/robot(1).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s
files/robot(1).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s


## Create a pipeline independent from master

In [58]:
!cat new-branch_image_tracer.yaml

pipeline:
  name: new-branch_image_tracer
description: A pipeline that performs image edge detection by using the OpenCV library.
input:
  pfs:
    repo: raw_videos_and_images
    glob: "/*.{png,jpg,jpeg}"
    branch: "new-branch"
transform:
  image: lbliii/image_tracer:1.0.8
  cmd:
    - python3
    - /image_tracer.py
    - --input
    - /pfs/raw_videos_and_images
    - --output
    - /pfs/out/
autoscaling: true
output_branch: "new-branch"


In [60]:
!pachctl list datum -f new-branch_image_tracer.yaml

ID                                                               FILES                                                                          STATUS TIME               
326dbd01a23fa9437ea3dfd3a8859495a8e3ac6e4aa5e127450e3d0c953bfce2 denisd-lab/raw_videos_and_images@6bbf1990fc3f46dcb63534da131c5d12:/liberty.png -      Less than a second 
88c2d6d260ff8c4a4ba3e21a97bdf628fbbaa1a6d088c500a7d3c11077255366 denisd-lab/raw_videos_and_images@6bbf1990fc3f46dcb63534da131c5d12:/robot1.png  -      Less than a second 
d49c6cbdbff2f1c2aabda69b0556eaf721764d9af6a172290e8fa33dcd9fa551 denisd-lab/raw_videos_and_images@6bbf1990fc3f46dcb63534da131c5d12:/robot.png   -      Less than a second 


In [61]:
!pachctl create pipeline -f new-branch_image_tracer.yaml

## Update the master branch using the new branch

In [ ]:
!pachctl create branch raw_videos_and_images@master --head raw_videos_and_images@new-branch

## Roll back the master branch

In [62]:
!pachctl create branch raw_videos_and_images@master --head raw_videos_and_images@master^

In [63]:
!pachctl delete pipeline new-branch_image_tracer
!pachctl delete branch raw_videos_and_images@new-branch

# Triggers

## Create a trigger

In [64]:
!pachctl create branch raw_videos_and_images@staging --head master

In [65]:
!pachctl list branch raw_videos_and_images

BRANCH  HEAD                             TRIGGER 
staging e9cf39575158423298cb8b5541ff5caf -       
master  e9cf39575158423298cb8b5541ff5caf -       


In [66]:
!pachctl create branch raw_videos_and_images@master --trigger staging --trigger-commits=3

In [67]:
!pachctl list branch raw_videos_and_images

BRANCH  HEAD                             TRIGGER               
staging e9cf39575158423298cb8b5541ff5caf -                     
master  e9cf39575158423298cb8b5541ff5caf staging on Commits(3) 


## Test the trigger

In [68]:
!pachctl put file raw_videos_and_images@staging:robot1.png -f "files/robot(1).jpg"

files/robot(1).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s
files/robot(1).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s


In [69]:
!pachctl put file raw_videos_and_images@staging:robot2.png -f "files/robot(2).jpg"

files/robot(2).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s
files/robot(2).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s
files/robot(2).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s


In [70]:
!pachctl put file raw_videos_and_images@staging:robot3.png -f "files/robot(3).jpg"

files/robot(3).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s
files/robot(3).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s
files/robot(3).jpg 123.23 KB / 123.23 KB [=========================] 0s 0.00 b/s


# Pachyderm SDK

## Install the Pachyderm SDK

In [71]:
!pip install pachyderm_sdk

  Obtaining dependency information for pachyderm_sdk from https://files.pythonhosted.org/packages/57/13/ac1bd4e7a10b73c9e0454273fec2a31c94bab4fa1f34ef62dd4820ec2b6c/pachyderm_sdk-2.7.6-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 30.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 108.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 128.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 204.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 169.1 MB/s eta 0:00:00
  Created wheel for grpclib: filename=grpclib-0.4.6-py3-none-any.whl size=76138 sha256=b7df7ad892f693adbba34b1731ca045527d18e14ecec99beeb1fcbdd9185db36
  Stored in

## Connect to the cluster

In [72]:
import pachyderm_sdk
from pachyderm_sdk.api import pfs, pps
from pachyderm_sdk.api.pfs import File, FileType

In [74]:
client = pachyderm_sdk.Client.from_config()

In [75]:
version = client.get_version()
print(version)

Version(major=2, minor=7, micro=5, git_commit='45809f13093271c2b3e8855a8e2884f9b93e6649', git_tree_modified='false', build_date='2023-10-18T14:10:51Z', go_version='go1.21.3', platform='amd64')


## List files from a repo

In [76]:
repo = "raw_videos_and_images"
branch = "master"

for file_info in client.pfs.walk_file(file=File.from_uri(f"{project_name}/{repo}@{branch}")):
    f_path = file_info.file.path
    print(f_path)

/
/cat-sleeping.MOV
/liberty.png
/robot.png
/robot1.png
/robot2.png
/robot3.png


## Create a repo

In [77]:
project = pfs.Project(name=project_name)
repo = pfs.Repo(name="sdk_repo", project=project)
branch = pfs.Branch.from_uri(f"{repo}@master")

try:
    client.pfs.create_repo(repo=repo, description="my first sdk-created repo")
    print("Repo creation successful.")
except Exception as e:
    print("Error creating project or repo:", e)
    exit(1)

Repo creation successful.


## Create a commit

In [78]:
try:
    with client.pfs.commit(branch=branch) as commit:
        with open("files/liberty.jpg", "rb") as source:
            commit.put_file_from_file(path="/liberty.png", file=source)
    print("Data loaded into the repo as a commit.")
except Exception as e:
    print("Error loading data into the repo:", e)
    exit(1)

Data loaded into the repo as a commit.


## Create a pipeline

In [79]:
try:
    input = pps.Input(pfs=pps.PfsInput(project=project.name, branch="master", repo=repo.name, glob="/*"))
    transform = pps.Transform(
        image="lbliii/image_tracer:1.0.8", 
        cmd=["python3", "/image_tracer.py", 
             "--input", "/pfs/sdk_repo",
             "--output", "/pfs/out/"]
    )

    pipeline = pps.Pipeline(name="sdk_image_tracer", project=project)

    client.pps.create_pipeline(pipeline=pipeline, input=input, transform=transform)
    print("Pipeline created successfully.")
except Exception as e:
    print("Error creating the pipeline:", e)
    exit(1)

Pipeline created successfully.


## Delete pipeline and repo

In [80]:
client.pps.delete_pipeline(pipeline=pipeline)

Empty()

In [81]:
client.pfs.delete_repo(repo=repo)

Empty()